In [1]:
import numpy as np
import pandas as pd
import dateutil.parser
import requests
import json

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np
from scipy.integrate import odeint

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#Helsingin Sanomat open data for comparison
url="https://w3qa5ydb4l.execute-api.eu-west-1.amazonaws.com/prod/finnishCoronaData"
res=requests.get(url)
hs_data = json.loads(res.content)
hs_df=pd.io.json.json_normalize(hs_data['confirmed'])
hs_df['one']=1
hs_df['total'] = hs_df.one.cumsum()

In [3]:
start = dateutil.parser.parse("2020-02-24")

# Total population, N.
N = 5.5e6
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0, D0 = 3, 0, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma, delta = 0.3, 1./14, 0.037/14.0
# A grid of time points (in days)
t = np.linspace(0, 500, 500)
R0=4.2
R0_after=2.4
day=60

contagious_days = 12

death_prob_per_day = 0.037/contagious_days
death_prob_ic = 0.15/contagious_days
death_prob_no_care = 0.3/contagious_days
ic_need = 0.06
ic_capacity=1000.

def betaT(t):
    r0_default = 3.0
    r0 = r0_default
    if t<30.:
        r0 = r0_default
    if t<37:
        r0 = r0_default * 0.5
    if t < 42:
        r0 = r0_default * 0.75

    return r0*gamma

def deltaI(i):
    if i*ic_need <  ic_capacity:
        return death_prob_per_day
    if i*ic_need > ic_capacity:
        return 

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R, D = y
    beta = betaT(t)
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I - delta * I
    dRdt = gamma * ( I - delta * I)
    
    if I*ic_need < ic_capacity:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
                + death_prob_ic * ic_need * I
               )
    else:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
        + death_prob_ic * ic_capacity
        + death_prob_no_care * (I*ic_need - ic_capacity)
        )
    return dSdt, dIdt, dRdt, dDdt

# Initial conditions vector
y0 = S0, I0, R0, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R, D = ret.T

df2=pd.DataFrame()
df2['t'] = t
df2['S'] = S
df2['I'] = I
df2['R'] = R
df2['D'] = D
df2['ts'] = pd.to_datetime(start.timestamp() + df2.t*86400, unit='s')
I_sc1 = I
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.ts, y=df2.I, name="Infected"),
      go.Scatter(x=df2.ts, y=df2.R, name="Recovered with immunity"),
      #go.Scatter(x=df2.ts, y=df2.D, name="Dead"),
      go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]
layout = go.Layout(
    title='Covid19, SIRD model' ,
    xaxis=dict(
        title='day',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        #type='log',
        title='people',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        ),
        #range = [1,500000],
    )
)
iplot({ 'data': data, "layout": layout }) 

In [4]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

start = dateutil.parser.parse("2020-02-24")

# Total population, N.
N = 5.5e6
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0, D0 = 3, 0, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma, delta = 0.3, 1./14, 0.037/14.0
# A grid of time points (in days)
t = np.linspace(0, 500, 500)
R0=4.2
R0_after=2.4
day=60

death_prob_per_day = 0.037/contagious_days
death_prob_ic = 0.15/contagious_days
death_prob_no_care = 0.3/contagious_days
ic_need = 0.06
ic_capacity=1000.

def betaT(t):
    r0_default = 3.0
    r0 = r0_default
    if t<30.:
        r0 = r0_default
    if t<37:
        r0 = r0_default * 0.75
    if t < 42:
        r0 = r0_default * 0.5

    return r0*gamma

def deltaI(i):
    if i*ic_need <  ic_capacity:
        return death_prob_per_day
    if i*ic_need > ic_capacity:
        return 

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R, D = y
    beta = betaT(t)
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I - delta * I
    dRdt = gamma * ( I - delta * I)
    
    if I*ic_need < ic_capacity:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
                + death_prob_ic * ic_need * I
               )
    else:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
        + death_prob_ic * ic_capacity
        + death_prob_no_care * (I*ic_need - ic_capacity)
        )
    return dSdt, dIdt, dRdt, dDdt

# Initial conditions vector
y0 = S0, I0, R0, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R, D = ret.T

df2=pd.DataFrame()
df2['t'] = t
df2['S'] = S
df2['I'] = I
I_sc2 = I
df2['R'] = R
df2['D'] = D
df2['ts'] = pd.to_datetime(start.timestamp() + df2.t*86400, unit='s')
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.ts, y=df2.I, name="Infected"),
      go.Scatter(x=df2.ts, y=df2.R, name="Recovered with immunity"),
      #go.Scatter(x=df2.ts, y=df2.D, name="Dead"),
      go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]

iplot({ 'data': data, "layout": layout }) 

In [5]:
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.t, y=I_sc1, name="Infected1"),
      go.Scatter(x=df2.t, y=I_sc2, name="Infected2"),
      #go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]

iplot({ 'data': data, "layout": layout }) 

In [6]:
df2

,t,S,I,R,D,ts
0,0.000000,5.499997e+06,3.000000,4.200000e+00,0.000000,2020-02-23 22:00:00.000000000
1,1.002004,5.499997e+06,3.101078,4.417735e+00,0.011151,2020-02-24 22:02:53.146292686
2,2.004008,5.499996e+06,3.205562,4.642807e+00,0.022677,2020-02-25 22:05:46.292585135
3,3.006012,5.499996e+06,3.313567,4.875462e+00,0.034592,2020-02-26 22:08:39.438877821
4,4.008016,5.499996e+06,3.425210,5.115955e+00,0.046908,2020-02-27 22:11:32.585170269
...,...,...,...,...,...,...
495,495.991984,3.703043e+05,0.002092,4.933600e+06,302601.478700,2021-07-03 21:48:27.414829731
496,496.993988,3.703043e+05,0.001970,4.933600e+06,302601.478708,2021-07-04 21:51:20.561122179
497,497.995992,3.703043e+05,0.001856,4.933600e+06,302601.478715,2021-07-05 21:54:13.707414865
498,498.997996,3.703043e+05,0.001748,4.933600e+06,302601.478721,2021-07-06 21:57:06.853707314
